# 04 – Modeling
## AdventureWorks | Capital Operativo

**Propósito**
Modelar métricas relativas y comparables que permitan evaluar la alineación
entre capital operativo (inventario) y generación de ventas.

**Principios**
- Modelos determinísticos (no ML)
- Explicables y trazables
- Enfocados en orientación (OODA), no en recomendación

In [1]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=127.0.0.1,1433;"
    "DATABASE=AdventureWorks2019;"
    "UID=sa;"
    "PWD=S1nt3lo_SQL_2026!;"
    "Encrypt=no;"
    "TrustServerCertificate=yes;"
)

In [3]:
# Productos
product = pd.read_sql("""
    SELECT
        ProductID,
        Name AS ProductName,
        ProductSubcategoryID,
        StandardCost
    FROM Production.Product
""", conn)

# Inventario (agregado)
inventory = pd.read_sql("""
    SELECT
        ProductID,
        SUM(Quantity) AS QuantityOnHand
    FROM Production.ProductInventory
    GROUP BY ProductID
""", conn)

# Ventas (agregado)
sales = pd.read_sql("""
    SELECT
        ProductID,
        SUM(LineTotal) AS SalesAmount
    FROM Sales.SalesOrderDetail
    GROUP BY ProductID
""", conn)

# Subcategoría
subcategory = pd.read_sql("""
    SELECT
        ProductSubcategoryID,
        ProductCategoryID,
        Name AS ProductSubcategory
    FROM Production.ProductSubcategory
""", conn)

# Categoría
category = pd.read_sql("""
    SELECT
        ProductCategoryID,
        Name AS ProductCategory
    FROM Production.ProductCategory
""", conn)

/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_56364/1348998874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql("""
/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_56364/1348998874.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  inventory = pd.read_sql("""
/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_56364/1348998874.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales = pd.read_sql("""
/var/folders/ry/4019pgsd4073m3w_k8vm8bqr0000gn/T/ipykernel_56364/

In [4]:
df = (
    product
    .merge(inventory, on="ProductID", how="left")
    .merge(sales, on="ProductID", how="left")
    .merge(subcategory, on="ProductSubcategoryID", how="left")
    .merge(category, on="ProductCategoryID", how="left")
)

In [5]:
df["QuantityOnHand"] = df["QuantityOnHand"].fillna(0)
df["SalesAmount"] = df["SalesAmount"].fillna(0)

df["InventoryValue"] = df["QuantityOnHand"] * df["StandardCost"]

In [6]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductID             504 non-null    int64  
 1   ProductName           504 non-null    object 
 2   ProductSubcategoryID  295 non-null    float64
 3   StandardCost          504 non-null    float64
 4   QuantityOnHand        504 non-null    float64
 5   SalesAmount           504 non-null    float64
 6   ProductCategoryID     295 non-null    float64
 7   ProductSubcategory    295 non-null    object 
 8   ProductCategory       295 non-null    object 
 9   InventoryValue        504 non-null    float64
dtypes: float64(6), int64(1), object(3)
memory usage: 39.5+ KB


## Definición del modelo

**Unidades de análisis**
- Producto (base)
- Agregación: Categoría

**Proxies**
- Capital operativo: InventoryValue
- Retorno observado: SalesAmount

**Supuestos**
- Inventario es stock observado
- Ventas son histórico agregado
- Comparación es relativa (porcentajes), no temporal

In [7]:
total_inventory = df["InventoryValue"].sum()
total_sales = df["SalesAmount"].sum()

In [8]:
df["pct_inventory"] = df["InventoryValue"] / total_inventory
df["pct_sales"] = df["SalesAmount"] / total_sales

In [14]:
model_category = (
    df.groupby("ProductCategory", as_index=False)
      .agg({
          "InventoryValue": "sum",
          "SalesAmount": "sum"
      })
)

model_category["pct_inventory"] = model_category["InventoryValue"] / total_inventory
model_category["pct_sales"] = model_category["SalesAmount"] / total_sales

In [17]:
model_category

,ProductCategory,InventoryValue,SalesAmount,pct_inventory,pct_sales
0,Accessories,8.821176e+04,1.272073e+06,0.004390,0.011580
1,Bikes,1.462365e+07,9.465117e+07,0.727810,0.861669
2,Clothing,1.381329e+05,2.120543e+06,0.006875,0.019305
3,Components,4.375837e+06,1.180259e+07,0.217783,0.107446


In [11]:
model_category["alignment_gap"] = (
    model_category["pct_sales"] - model_category["pct_inventory"]
)

In [12]:
model_category[["pct_inventory","pct_sales","alignment_gap"]].sum()

pct_inventory    0.956858
pct_sales        1.000000
alignment_gap    0.043142
dtype: float64

In [21]:
model_category.columns

Index(['ProductCategory', 'InventoryValue', 'SalesAmount', 'pct_inventory',
       'pct_sales'],
      dtype='object')

In [22]:
model_category["alignment_gap"] = (
    model_category["pct_sales"] - model_category["pct_inventory"]
)

In [23]:
model_category

,ProductCategory,InventoryValue,SalesAmount,pct_inventory,pct_sales,alignment_gap
0,Accessories,8.821176e+04,1.272073e+06,0.004390,0.011580,0.007190
1,Bikes,1.462365e+07,9.465117e+07,0.727810,0.861669,0.133859
2,Clothing,1.381329e+05,2.120543e+06,0.006875,0.019305,0.012430
3,Components,4.375837e+06,1.180259e+07,0.217783,0.107446,-0.110336


In [24]:
presentation_table = model_category[[
    "ProductCategory",
    "pct_inventory",
    "pct_sales",
    "alignment_gap"
]].sort_values("alignment_gap", ascending=False)

presentation_table

,ProductCategory,pct_inventory,pct_sales,alignment_gap
1,Bikes,0.727810,0.861669,0.133859
2,Clothing,0.006875,0.019305,0.012430
0,Accessories,0.004390,0.011580,0.007190
3,Components,0.217783,0.107446,-0.110336


## Cierre — Modeling

Se construyeron métricas relativas y señales de alineación
entre capital operativo y ventas a nivel categoría.

El siguiente paso (`05_evaluation.ipynb`) evaluará:
- patrones
- concentraciones
- desalineaciones relevantes
sin emitir recomendaciones.

In [30]:
df.columns

Index(['ProductID', 'ProductName', 'ProductSubcategoryID', 'StandardCost',
       'QuantityOnHand', 'SalesAmount', 'ProductCategoryID',
       'ProductSubcategory', 'ProductCategory', 'InventoryValue',
       'pct_inventory', 'pct_sales'],
      dtype='object')

In [31]:
total_inventory = df["InventoryValue"].sum()
total_sales = df["SalesAmount"].sum()

total_inventory, total_sales

(np.float64(20092679.1712), np.float64(109846381.39988801))

In [32]:
model_subcategory = (
    df.groupby(
        ["ProductCategory", "ProductSubcategory"],
        as_index=False
    )
    .agg({
        "InventoryValue": "sum",
        "SalesAmount": "sum"
    })
)

In [33]:
model_subcategory.head()
model_subcategory.columns

Index(['ProductCategory', 'ProductSubcategory', 'InventoryValue',
       'SalesAmount'],
      dtype='object')

In [34]:
model_subcategory["pct_inventory"] = (
    model_subcategory["InventoryValue"] / total_inventory
)

model_subcategory["pct_sales"] = (
    model_subcategory["SalesAmount"] / total_sales
)

In [35]:
model_subcategory[["pct_inventory", "pct_sales"]].describe()

,pct_inventory,pct_sales
count,37.000000,37.000000
mean,0.025861,0.027027
std,0.071159,0.085510
min,0.000000,0.000000
25%,0.000185,0.000466
50%,0.000766,0.001343
75%,0.006383,0.003765
max,0.344771,0.399735


In [36]:
model_subcategory["alignment_gap"] = (
    model_subcategory["pct_sales"]
    - model_subcategory["pct_inventory"]
)

In [37]:
model_subcategory.head()

,ProductCategory,ProductSubcategory,InventoryValue,SalesAmount,pct_inventory,pct_sales,alignment_gap
0,Accessories,Bike Racks,0.0000,237096.156000,0.000000,0.002158,0.002158
1,Accessories,Bike Stands,8563.1040,39591.000000,0.000426,0.000360,-0.000066
2,Accessories,Bottles and Cages,2635.7472,64274.793327,0.000131,0.000585,0.000454
3,Accessories,Cleaners,107.0388,18406.972080,0.000005,0.000168,0.000162
4,Accessories,Fenders,887.8140,46619.580000,0.000044,0.000424,0.000380


In [38]:
model_subcategory = model_subcategory.sort_values(
    "alignment_gap", ascending=False
)

In [39]:
model_subcategory

,ProductCategory,ProductSubcategory,InventoryValue,SalesAmount,pct_inventory,pct_sales,alignment_gap
12,Bikes,Mountain Bikes,4.771084e+06,3.644544e+07,0.237454,0.331786,0.094332
13,Bikes,Road Bikes,6.927376e+06,4.390944e+07,0.344771,0.399735,0.054964
33,Components,Road Frames,2.873105e+05,3.851351e+06,0.014299,0.035061,0.020762
35,Components,Touring Frames,0.000000e+00,1.642328e+06,0.000000,0.014951,0.014951
18,Clothing,Jerseys,4.844514e+04,7.522594e+05,0.002411,0.006848,0.004437
5,Accessories,Helmets,1.083546e+04,4.840485e+05,0.000539,0.004407,0.003867
0,Accessories,Bike Racks,0.000000e+00,2.370962e+05,0.000000,0.002158,0.002158
19,Clothing,Shorts,3.760633e+04,4.136005e+05,0.001872,0.003765,0.001894
17,Clothing,Gloves,8.374720e+03,2.435120e+05,0.000417,0.002217,0.001800
22,Clothing,Vests,1.538935e+04,2.594884e+05,0.000766,0.002362,0.001596


In [41]:
presentation_subcategory = model_subcategory[[
    "ProductCategory",
    "ProductSubcategory",
    "pct_inventory",
    "pct_sales",
    "alignment_gap"
]]

In [42]:
presentation_subcategory.head()

,ProductCategory,ProductSubcategory,pct_inventory,pct_sales,alignment_gap
12,Bikes,Mountain Bikes,0.237454,0.331786,0.094332
13,Bikes,Road Bikes,0.344771,0.399735,0.054964
33,Components,Road Frames,0.014299,0.035061,0.020762
35,Components,Touring Frames,0.000000,0.014951,0.014951
18,Clothing,Jerseys,0.002411,0.006848,0.004437


In [43]:
presentation_subcategory = presentation_subcategory.copy()

for col in ["pct_inventory", "pct_sales", "alignment_gap"]:
    presentation_subcategory[col] *= 100

presentation_subcategory

,ProductCategory,ProductSubcategory,pct_inventory,pct_sales,alignment_gap
12,Bikes,Mountain Bikes,23.745384,33.178557,9.433172
13,Bikes,Road Bikes,34.477114,39.973495,5.496380
33,Components,Road Frames,1.429926,3.506124,2.076198
35,Components,Touring Frames,0.000000,1.495113,1.495113
18,Clothing,Jerseys,0.241108,0.684829,0.443720
5,Accessories,Helmets,0.053927,0.440660,0.386732
0,Accessories,Bike Racks,0.000000,0.215843,0.215843
19,Clothing,Shorts,0.187164,0.376526,0.189362
17,Clothing,Gloves,0.041680,0.221684,0.180004
22,Clothing,Vests,0.076592,0.236228,0.159637
